<b>Joining Operations</b>

In [10]:
import pprint
import json
import pymongo
from pymongo import MongoClient
from datetime import datetime
client = MongoClient('mongodb://localhost:27017/')
#client = MongoClient('localhost', 27017)

In [59]:
db=client.warehouse

In [60]:
"""db.inventory.insert([
   { "_id" : 1, "sku" : "almonds", "description": "product 1", "instock" : 120 },
   { "_id" : 2, "sku" : "bread", "description": "product 2", "instock" : 80 },
   { "_id" : 3, "sku" : "cashews", "description": "product 3", "instock" : 60 },
   { "_id" : 4, "sku" : "pecans", "description": "product 4", "instock" : 70 },
   { "_id" : 5, "sku": "almonds", "description": "Incomplete" },
   { "_id" : 6 }
])"""

'db.inventory.insert([\n   { "_id" : 1, "sku" : "almonds", "description": "product 1", "instock" : 120 },\n   { "_id" : 2, "sku" : "bread", "description": "product 2", "instock" : 80 },\n   { "_id" : 3, "sku" : "cashews", "description": "product 3", "instock" : 60 },\n   { "_id" : 4, "sku" : "pecans", "description": "product 4", "instock" : 70 },\n   { "_id" : 5, "sku": "almonds", "description": "Incomplete" },\n   { "_id" : 6 }\n])'

In [123]:
db.inventory.delete_one({"$and":[{"sku":"almonds"},{"instock":1000}]})

In [124]:
list(db.inventory.find())

[{'_id': 2, 'sku': 'bread', 'description': 'product 2', 'instock': 580},
 {'_id': 3, 'sku': 'cashews', 'description': 'product 3', 'instock': 560},
 {'_id': 4, 'sku': 'pecans', 'description': 'product 4', 'instock': 570},
 {'_id': 6, 'instock': 500}]

In [92]:
"""db.orders.insert([
   { "_id" : 1, "item" : "almonds", "price" : 12, "quantity" : 2 },
   { "_id" : 2, "item" : "pecans", "price" : 20, "quantity" : 1 },
   { "_id" : 3  }
])"""

'db.orders.insert([\n   { "_id" : 1, "item" : "almonds", "price" : 12, "quantity" : 2 },\n   { "_id" : 2, "item" : "pecans", "price" : 20, "quantity" : 1 },\n   { "_id" : 3  }\n])'

<b>Perform a Single Equality Join with $lookup</b>

In [107]:
list(db.orders.aggregate([{"$lookup":{
                     "from":"inventory",
                     "localField":"item",
                     "foreignField":"sku",
                      "as": "NewData"
}},{"$project":{"_id":0}}]))

[{'item': 'almonds',
  'price': 12,
  'quantity': 2,
  'NewData': [{'_id': 1,
    'sku': 'almonds',
    'description': 'product 1',
    'instock': 120},
   {'_id': 5, 'sku': 'almonds', 'description': 'Incomplete'}]},
 {'item': 'pecans',
  'price': 20,
  'quantity': 1,
  'NewData': [{'_id': 4,
    'sku': 'pecans',
    'description': 'product 4',
    'instock': 70}]},
 {'NewData': [{'_id': 6}]}]

<b>Multiple Join Conditions with $lookup</b>

In [72]:
"""db.Orders.insert([
  { "_id" : 1, "item" : "almonds", "price" : 12, "ordered" : 2 },
  { "_id" : 2, "item" : "pecans", "price" : 20, "ordered" : 1 },
  { "_id" : 3, "item" : "cookies", "price" : 10, "ordered" : 60 }
])"""

'db.Orders.insert([\n  { "_id" : 1, "item" : "almonds", "price" : 12, "ordered" : 2 },\n  { "_id" : 2, "item" : "pecans", "price" : 20, "ordered" : 1 },\n  { "_id" : 3, "item" : "cookies", "price" : 10, "ordered" : 60 }\n])'

In [126]:
db.orders.aggregate([{"$match":{"$expr":{"$gte":["$price","$ordered"]}}},{"$group":{"_id":"item","Sum_ordered":{"$sum":"$price"}}}])

[{'_id': 'item', 'Sum_ordered': 32}]

In [71]:
"""db.warehouses.insert([
  { "_id" : 1, "stock_item" : "almonds", "warehouse": "A", "instock" : 120 },
  { "_id" : 2, "stock_item" : "pecans", "warehouse": "A", "instock" : 80 },
  { "_id" : 3, "stock_item" : "almonds", "warehouse": "B", "instock" : 60 },
  { "_id" : 4, "stock_item" : "cookies", "warehouse": "B", "instock" : 40 },
  { "_id" : 5, "stock_item" : "cookies", "warehouse": "A", "instock" : 80 }
])"""

'db.warehouses.insert([\n  { "_id" : 1, "stock_item" : "almonds", "warehouse": "A", "instock" : 120 },\n  { "_id" : 2, "stock_item" : "pecans", "warehouse": "A", "instock" : 80 },\n  { "_id" : 3, "stock_item" : "almonds", "warehouse": "B", "instock" : 60 },\n  { "_id" : 4, "stock_item" : "cookies", "warehouse": "B", "instock" : 40 },\n  { "_id" : 5, "stock_item" : "cookies", "warehouse": "A", "instock" : 80 }\n])'

In [135]:
db.orders.aggregate([
   {"$lookup": {
           "from": "warehouses",
           "let": {"order_item": "$item", "order_qty": "$ordered" },
           "pipeline": [
              {"$match":
                 {"$expr":
                    {"$and":
                       [
                         {"$eq": [ "$stock_item",  "$$order_item" ] },
                         {"$gte": [ "$instock", "$$order_qty" ] }
                       ]
                    }
                 }
              }
               ,{"$project":{ "stock_item": 0, "_id": 0 } }],
           "as": "stockdata"
         }
    },{"$out":"reviewedorder"}
])

In [136]:
list(db.reviewedorder.find())

[{'_id': 1,
  'item': 'almonds',
  'price': 12,
  'quantity': 2,
  'stockdata': [{'warehouse': 'A', 'instock': 120},
   {'warehouse': 'B', 'instock': 60}]},
 {'_id': 2,
  'item': 'pecans',
  'price': 20,
  'quantity': 1,
  'stockdata': [{'warehouse': 'A', 'instock': 80}]},
 {'_id': 3, 'stockdata': []}]

In [46]:
list(db.orders.aggregate([{"$lookup":{
                   "from":"warehouses",
                    "let":{"orderitem":"$item","order_qty":"$ordered"},
                     "pipeline":[{
                         "$match":{
                             "$expr":{"$and":[{
                                 "$eq":["$stock_item","$$orderitem"]},
                                  {"$gte":["$instock","$$order_qty"]}
                             ]
                             }
                         }
                     }
                     ,{"$project":{"_id":0,"stock_item":0}}],
                   "as":"Stockdata"
}}]))

[{'_id': 1,
  'item': 'almonds',
  'price': 12,
  'quantity': 2,
  'Stockdata': [{'warehouse': 'A', 'instock': 120},
   {'warehouse': 'B', 'instock': 60}]},
 {'_id': 2,
  'item': 'pecans',
  'price': 20,
  'quantity': 1,
  'Stockdata': [{'warehouse': 'A', 'instock': 80}]},
 {'_id': 3, 'Stockdata': []}]

<b>Use $lookup with mergeObjects</b>

In [70]:
"""db.items.insert([
  { "_id" : 1, "item" : "almonds", "description": "almond clusters", "instock" : 120 },
  { "_id" : 2, "item" : "bread", "description": "raisin and nut bread", "instock" : 80 },
  { "_id" : 3, "item" : "pecans", "description": "candied pecans", "instock" : 60 }
])"""

'db.items.insert([\n  { "_id" : 1, "item" : "almonds", "description": "almond clusters", "instock" : 120 },\n  { "_id" : 2, "item" : "bread", "description": "raisin and nut bread", "instock" : 80 },\n  { "_id" : 3, "item" : "pecans", "description": "candied pecans", "instock" : 60 }\n])'

In [59]:
list(db.orders.aggregate([
   { "$lookup": {
         "from": "items",
         "localField": "item",    # field in the orders collection
         "foreignField": "item",  # field in the items collection
         "as": "fromItems"
      }
   },
   {"$replaceRoot": {"newRoot": {"$mergeObjects": [ {"$arrayElemAt": [ "$fromItems", 0 ] }, "$$ROOT" ] } }
   },
   {"$project": { "fromItems": 0 } }
]))

[{'_id': 1,
  'item': 'almonds',
  'description': 'almond clusters',
  'instock': 120,
  'price': 12,
  'quantity': 2},
 {'_id': 2,
  'item': 'pecans',
  'description': 'candied pecans',
  'instock': 60,
  'price': 20,
  'quantity': 1},
 {'_id': 3}]

<b>Use $lookup with an Array</b>

In [69]:
""""db.classes.insert( [
   { "_id": 1, "title": "Reading is ...", "enrollmentlist": [ "giraffe2", "pandabear", "artie" ], "days": ["M", "W", "F"] },
   { "_id": 2, "title": "But Writing ...", "enrollmentlist": [ "giraffe1", "artie" ], "days": ["T", "F"] }
])"""

'"db.classes.insert( [\n   { "_id": 1, "title": "Reading is ...", "enrollmentlist": [ "giraffe2", "pandabear", "artie" ], "days": ["M", "W", "F"] },\n   { "_id": 2, "title": "But Writing ...", "enrollmentlist": [ "giraffe1", "artie" ], "days": ["T", "F"] }\n])'

In [68]:
"""db.members.insert_many( [
   { "_id": 1, "name": "artie", "joined": "new Date()", "status": "A" },
   { "_id": 2, "name": "giraffe","joined": "new Date()", "status": "D" },
   { "_id": 3, "name": "giraffe1", "joined": "new Date()", "status": "A" },
   { "_id": 4, "name": "panda", "joined": "new Date()", "status": "A" },
   { "_id": 5, "name": "pandabear", "joined": "new Date()", "status": "A" },
   { "_id": 6, "name": "giraffe2", "joined": "new Date()", "status": "D" }
])"""

'db.members.insert_many( [\n   { "_id": 1, "name": "artie", "joined": "new Date()", "status": "A" },\n   { "_id": 2, "name": "giraffe","joined": "new Date()", "status": "D" },\n   { "_id": 3, "name": "giraffe1", "joined": "new Date()", "status": "A" },\n   { "_id": 4, "name": "panda", "joined": "new Date()", "status": "A" },\n   { "_id": 5, "name": "pandabear", "joined": "new Date()", "status": "A" },\n   { "_id": 6, "name": "giraffe2", "joined": "new Date()", "status": "D" }\n])'

In [62]:
results=db.classes.aggregate([
   {"$lookup":
         {"from": "members",
            "localField": "enrollmentlist",
            "foreignField": "name",
            "as": "enrollee_info"
        }
   }
])



In [67]:
list(results)

[{'_id': 1,
  'title': 'Reading is ...',
  'enrollmentlist': ['giraffe2', 'pandabear', 'artie'],
  'days': ['M', 'W', 'F'],
  'enrollee_info': [{'_id': 1,
    'name': 'artie',
    'joined': 'new Date()',
    'status': 'A'},
   {'_id': 5, 'name': 'pandabear', 'joined': 'new Date()', 'status': 'A'},
   {'_id': 6, 'name': 'giraffe2', 'joined': 'new Date()', 'status': 'D'}]},
 {'_id': 2,
  'title': 'But Writing ...',
  'enrollmentlist': ['giraffe1', 'artie'],
  'days': ['T', 'F'],
  'enrollee_info': [{'_id': 1,
    'name': 'artie',
    'joined': 'new Date()',
    'status': 'A'},
   {'_id': 3, 'name': 'giraffe1', 'joined': 'new Date()', 'status': 'A'}]}]

In [49]:
list(db.orders.aggregate([{"$lookup":{
                      "from":"inventory",
                       "localField":"item",
                       "foreignField":"sku",
                        "as":"DataArray"
}}]))

[{'_id': 1,
  'item': 'almonds',
  'price': 12,
  'quantity': 2,
  'DataArray': [{'_id': 1,
    'sku': 'almonds',
    'description': 'product 1',
    'instock': 120},
   {'_id': 5, 'sku': 'almonds', 'description': 'Incomplete'}]},
 {'_id': 2,
  'item': 'pecans',
  'price': 20,
  'quantity': 1,
  'DataArray': [{'_id': 4,
    'sku': 'pecans',
    'description': 'product 4',
    'instock': 70}]},
 {'_id': 3, 'DataArray': [{'_id': 6}]}]

In [56]:
list(db.orders.aggregate([{"$lookup":{
                     "from":'warehouses',
                      "let":{"order_items":"$item","qty_ordered":"$ordered"},
                       "pipeline":[{
                                    "$match":{
                                        "$expr":{
                                            "$and":[{
                                                "$eq":["$stock_item","$$order_items"]},
                                                {"$gte":["$instocK","$$qty_ordered"]}
                                            ]}}},{"$project":{"_id":0,"stock_item":0}}],"as":"New array"}}]))
                                        
        

[{'_id': 1,
  'item': 'almonds',
  'price': 12,
  'quantity': 2,
  'New array': [{'warehouse': 'A', 'instock': 120},
   {'warehouse': 'B', 'instock': 60}]},
 {'_id': 2,
  'item': 'pecans',
  'price': 20,
  'quantity': 1,
  'New array': [{'warehouse': 'A', 'instock': 80}]},
 {'_id': 3, 'New array': []}]